In [13]:
# main.ipynb
# Archivo base en el que se maneja la experimentación del diseño metodológico.
# Objetivo general: Utilizar los datos publicos (ubicados en la carpeta /data/)
#     de municipios en los que existe información para diseñar una estrategia
#     para distribuir alimentos en el territorio nacional.
# Objetivos específicos:

#     ---pronostico_poblacional---
#     [IA]1. Determinar por municipio que herramienta es mejor para pronosticar
#            la población en 10 años a partir del año actual.
#               a. Regresión Lineal Multiple || Multiple Linear Regression.
#               b. Arboles de Regresión || Regression Tree.
#               c. Máquinas de Vectores de Soporte || Support Vector Machine.
#               d. Bosques Aleatorios || Random Forest Regression.
#               e. Redes neurales || Deep Learning.
#               f. Regresión Tradicional.
#     ---capacidad_y_costo---
#     [IO]2. Determinar las capacidades y costos de almacenamiento de alimentos
#            en cada municipio utilizando los datos del pronóstico, los datos
#            abiertos y consideraciones de los analístas.
#     ---pronosticar_capacidad_y_costo---
#     [IA]3. Entrenar un clasificador desde los datos de capacidades y costos
#            de almacenamiento por municipio.
#               a. Arboles de Decision || Decision Tree.
#               b. Análisis Discriminante Lineal || Linear Discriminant Analysis.
#               c. Regresión Logística || Logistic Regression.
#               d. Máquinas de Vectores de Soporte || Support Vector Machine.
#               e. Redes Neurales || Deep Learning.
#               f. Análisis de frecuencias. [Aún está en desarrollo la idea]
#     ---cantidad_de_clusteres---
#     [IO]4. Proponer una cantidad de clusteres que permitan disminuir el costo
#            computacional del algoritmo optimizador y compararlo con los métodos
#            tradicionales.
#     ---generar_clusteres---
#     [IA]5. Dividir los municipios en clústeres, rectificando que es viable
#            satisfacer la demanda de alimentos con la capacidad instalada del
#            municipio.
#               a. k-means.
#               b. Mapa Autoorganizado || Self-Organizing Map.
#               c. Agrupamiento Jerárquico || Agglomerative Clustering.
#               d. DBSCAN.
#     ---solucionar_cflp---
#     [IO]6. Resolver el cflp para diferentes escenarios.
#               a. Solución ingenua (todos los municipios tienen un centro de
#                  distribución).
#               b. Datos completos sin clusterizar.
#               c. Dividido por clústers.
#


In [14]:
# Manejo básico de archivos
import os, sys, warnings

# Registro del tiempo
import time

# Manejo de datos
import pandas as pd
import numpy as np

# Funciones personalizadas de /funciones/funciones.py
sys.path.append("funciones")
from funciones import *

# regresar al directorio principal
sys.path.append("..")

# Verificación de datos geográficos
import geopy.distance


In [15]:
# Funciones
import dis


def crear_estructura_de_archivos():
    """
    definición de la estructura de archivos

    carpeta_raiz
        /data
            matriz-de-costos.csv (COP/(KM)) *
            matriz-de-distancias.csv **
            municipios.csv ** (cód, nombres, lat, lon, e historico de población)
            opciones-de-almacenes.xlsx (sheet_name: divipol del municipio)
        /funciones
            funciones.py
        /resultados
        1. pronostico_poblacional.csv
        2. capacidad_y_costo.csv
        3. cantidad_de_clusteres.csv
        4. generacion_de_clusteres.csv
        5. solucionar_cflp.csv
            /tablas
                /pronostico_poblacional
                    info-completa.csv
                    info-imperfecta.csv
                /capacidad_y_costo
                    info-completa.csv
                    info-imperfecta.csv
            /graficos
                /pronostico_poblacional
                    info-completa.png
                    info-imperfecta.png
                /capacidad_y_costo
                    info-completa.png
                    info-imperfecta.png
            /mapas
                /completo
                    1. ingenua.png
                    2. sin_clusterizar.png
                    3. clusterizada-k-means.png
                    4. clusterizada-mapa_autoorganizado.png
                    5. clusterizada-agrupamiento_jerarquico.png
                /imperfecto
                    1. ingenua.png
                    2. sin_clusterizar.png
                    3. clusterizada-k-means.png
                    4. clusterizada-mapa_autoorganizado.png
                    5. clusterizada-agrupamiento_jerarquico.png
            /logs
                /2. capacidad_y_costo
                    info-completa.log
                    info-imperfecta.log
                /4. cantidad_de_clusteres
                    info-completa.log
                    info-imperfecta.log
                /6. solucionar_cflp
                    /1. ingenua
                        info-completa.log
                        info-imperfecta.log
                    /2. sin_clusterizar
                        info-completa.log
                        info-imperfecta.log
                    /3. clusterizada
                        /1. k-means
                            /info-completa
                                [Archivo por cluster].log
                            /info-imperfecta
                                [Archivo por cluster].log
                        /2. mapa_autoorganizado
                            /info-completa
                                [Archivo por cluster].log
                            /info-imperfecta
                                [Archivo por cluster].log
                        /3. agrupamiento_jerarquico
                            /info-completa
                                [Archivo por cluster].log
                            /info-imperfecta
                                [Archivo por cluster].log
        * Datos en los que existe información para los municipios
        ** Datos para todos los municipios de colombia
    """
    try:
        # Crear la estructura de archivos
        os.makedirs("data", exist_ok=True)
        os.makedirs("funciones", exist_ok=True)
        os.makedirs("resultados", exist_ok=True)
        os.makedirs("resultados/tablas", exist_ok=True)
        os.makedirs("resultados/tablas/pronostico_poblacional", exist_ok=True)
        os.makedirs("resultados/tablas/capacidad_y_costo", exist_ok=True)
        os.makedirs("resultados/graficos", exist_ok=True)
        os.makedirs("resultados/graficos/pronostico_poblacional", exist_ok=True)
        os.makedirs("resultados/graficos/capacidad_y_costo", exist_ok=True)
        os.makedirs("resultados/mapas", exist_ok=True)
        os.makedirs("resultados/mapas/completo", exist_ok=True)
        os.makedirs("resultados/mapas/imperfecto", exist_ok=True)
        os.makedirs("resultados/logs", exist_ok=True)
        os.makedirs("resultados/logs/2. capacidad_y_costo", exist_ok=True)
        os.makedirs("resultados/logs/4. cantidad_de_clusteres", exist_ok=True)
        os.makedirs("resultados/logs/6. solucionar_cflp", exist_ok=True)
        os.makedirs("resultados/logs/6. solucionar_cflp/1. ingenua", exist_ok=True)
        os.makedirs(
            "resultados/logs/6. solucionar_cflp/2. sin_clusterizar", exist_ok=True
        )
        os.makedirs("resultados/logs/6. solucionar_cflp/3. clusterizada", exist_ok=True)
        os.makedirs(
            "resultados/logs/6. solucionar_cflp/3. clusterizada/1. k-means",
            exist_ok=True,
        )
        os.makedirs(
            "resultados/logs/6. solucionar_cflp/3. clusterizada/2. mapa_autoorganizado",
            exist_ok=True,
        )
        os.makedirs(
            "resultados/logs/6. solucionar_cflp/3. clusterizada/3. agrupamiento_jerarquico",
            exist_ok=True,
        )
    except Exception as e:
        print(f"Error: {e}")
        return False


def cargar_datos():
    """
    Cargar los datos de los archivos .csv y .xlsx

    Returns
    -------
    matriz_de_costos : pd.DataFrame
        Matriz de costos de transporte entre municipios sacados del SiceTac
    matriz_de_distancias : pd.DataFrame
        Matriz de distancias entre todos los municipios de Colombia por carretera con OpenStreetMap
    municipios : pd.DataFrame
        Información de los municipios de Colombia con historico de población 1985-2023
    opciones_de_almacenes : dict
        Opciones de los mejores 21 almacenes por departamento {divipol: pd.DataFrame}
    """
    try:
        # Cargar los datos
        matriz_de_costos = pd.read_csv("data/matriz-de-costos.csv", index_col=0)
        matriz_de_distancias = pd.read_csv("data/matriz-de-distancias.csv", index_col=0)
        # pasar los nombres de las columnas a enteros
        matriz_de_distancias.columns = matriz_de_distancias.columns.astype(int)
        matriz_de_distancias.index = matriz_de_distancias.index.astype(int)
        # Convertir las medidas de metros a kilometros
        matriz_de_distancias = matriz_de_distancias / 1000
        municipios = pd.read_csv("data/municipios.csv", index_col=3)
        opciones_de_almacenes = {}
        xls = pd.ExcelFile("data/opciones-de-almacenes.xlsx")
        for sheet_name in xls.sheet_names:
            # if the sheet is not empty
            if not xls.parse(sheet_name).empty:
                opciones_de_almacenes[sheet_name] = pd.read_excel(xls, sheet_name)
        return (
            matriz_de_costos,
            matriz_de_distancias,
            municipios,
            opciones_de_almacenes,
        )
    except Exception as e:
        print(f"Error: {e}")
        return tuple([None] * 4)


def alistar_datos_completos(
    matriz_de_costos,
    matriz_de_distancias,
    municipios,
    opciones_de_almacenes,
    comida_per_capita,
    densidad_de_alimentos,
):
    """
    Alistar los datos que de los que se tiene información completa, es decir,
    los municipios para los cuales se tienen datos en la matriz de costos

    Parametros
    ----------
    matriz_de_costos : pd.DataFrame
        Matriz de costos de transporte entre municipios sacados del SiceTac
    matriz_de_distancias : pd.DataFrame
        Matriz de distancias entre todos los municipios de Colombia por carretera con OpenStreetMap
    municipios : pd.DataFrame
        Información de los municipios de Colombia con historico de población 1985-2023
    opciones_de_almacenes : dict
        Opciones de los mejores 21 almacenes por departamento {divipol: pd.DataFrame}
    comida_per_capita : float
        Cantidad de comida necesaria por persona
    densidad_de_alimentos : float
        Densidad de los alimentos

    Returns
    -------
    matriz_de_costos_final : pd.DataFrame
        Matriz de costos de transporte entre municipios con información completa
    matriz_de_distancias_final : pd.DataFrame
        Matriz de distancias entre todos los municipios de Colombia con información completa
    municipios_final : pd.DataFrame
        Información de los municipios de Colombia con historico de población 1985-2023 con información completa
    seleccion_de_almacenes_final : pd.DataFrame
        Selección de los almacenes que cumplen con la demanda de almacenamiento

    Proceso
    -------
    # Matriz de costos
    1. Rectificar que no existan indices o nombres de columna duplicados
    2. Rectificar que no existan valores nulos
    3. Rectificar que no existan valores negativos
    4. Rectificar que no existan valores iguales a cero fuera de la diagonal principal
    5. Rectificar que la matriz sea simétrica
    6. Rectificar que la diagonal principal sea cero
    # Matriz de distancias
    1. Eliminar filas y columnas que no estén en la matriz de costos
    2. Rectificar que no existan indices o nombres de columna duplicados
    3. Rectificar que no existan valores nulos
        3.1 Si existen valores nulos, eliminar la fila
        3.2 Rectificar que las las columnas tengan los mismos nombres que las filas
    4. Rectificar que no existan valores negativos
    5. Rectificar que no existan valores iguales a cero fuera de la diagonal principal
    6. Rectificar que la matriz sea simétrica
    7. Rectificar que la diagonal principal sea cero
    8. utilizar el valor máximo entre la distancia entre los municipios y la distancia en línea recta
    # Municipios
    1. Eliminar filas que no estén en la matriz de costos
    2. tomar solo las columnas de interés
        a. divipola [index]
        b. nombre del municipio [municipio]
        c. nombre del departamento [departamento]
        d. latitud [lat]
        e. longitud [lon]
        f. población [1985-2023]
    # Opciones de almacenes
    1. Eliminar las opciones que no estén en la matriz de costos
    2. Encontrar por municipio que tantos m2 de almacenamiento se necesitan
    3. Revisar si existe un almacen que cumpla la demanda por un factor de 1.5
        3.1. Si no existe, revisar si la suma de los almacenes cumple la demanda por un factor de 1.5
            3.1.1. Entre los almacenes cumplen más de la demanda, ordenar los almacenes por almacenamiento
                   de mayor a menor y sumar los primeros hasta cumplir la demanda y crear un nuevo almacen
                   con la suma de almacenamiento y la suma de costos
        3.2. Si no existe, no reportar el municipio
    4. Rankear los precios de los almacenes
    5. Rankear la capacidad de los almacenes
    6. Calcular (Rankeo de precios * 0.2 + Rankeo de capacidad * 0.8)
    7. Seleccionar el mejor almacen por departamento
    8. Retornar una lista con los datos [divipol, location -> ubicacion, area, price -> precio]
    """
    try:
        # Matriz de costos
        matriz_de_costos_final = procesar_matriz_de_costos_completos(matriz_de_costos)
        print("Matriz de costos completada")

        # Matriz de distancias
        matriz_de_distancias_final = procesar_matriz_de_distancias_completas(
            matriz_de_distancias, municipios, matriz_de_costos_final
        )
        print("Matriz de distancias completada")

        # Municipios
        municipios_final = procesar_municipios_completos(
            municipios, matriz_de_costos_final
        )
        print("Municipios completados")

        # Opciones de almacenes
        seleccion_de_almacenes_final = procesar_opciones_de_almacenes(
            opciones_de_almacenes,
            comida_per_capita,
            densidad_de_alimentos,
            matriz_de_costos_final,
            municipios_final,
        )
        print("Opciones de almacenes completadas")

        return (
            matriz_de_costos_final,
            matriz_de_distancias_final,
            municipios_final,
            seleccion_de_almacenes_final,
        )
    except Exception as e:
        print(f"Error: {e}")
        return tuple([None] * 4)


def procesar_opciones_de_almacenes(
    opciones_de_almacenes,
    comida_per_capita,
    densidad_de_alimentos,
    matriz_de_costos_final,
    municipios_final,
):
    opciones_de_almacenes_final = pd.DataFrame(
        columns=["divipol", "ubicacion", "area", "capacidad", "precio"]
    )
    for divipol, opciones in opciones_de_almacenes.items():
        divipol = int(divipol)
        # 1. Eliminar las opciones que no estén en la matriz de costos
        if divipol not in matriz_de_costos_final.index:
            print(f"     No se encontró información para el municipio {divipol}")
            continue
            # 2. Encontrar por municipio que tantos m2 de almacenamiento se necesitan
        demanda = municipios_final.loc[divipol, "2023"] * comida_per_capita
        # 3. Revisar si existe un almacen que cumpla la demanda por un factor de 1.5
        opciones["capacidad"] = (
            opciones["area"] * densidad_de_alimentos * 5
        )  # 5 metros de altura
        opciones["cumple_demanda"] = opciones["capacidad"] >= demanda * 1.5

        opciones = opciones[opciones["cumple_demanda"]].copy()
        if opciones["cumple_demanda"].sum() == 0:
            print(
                f"     No se encontró un almacen que cumpla la demanda para {divipol}"
            )
            continue

        if demanda == 0:
            print(f"     La demanda para el municipio {divipol} es cero")
            continue

        # 4. Rankear los precios de los almacenes
        opciones["rank_precio"] = opciones["price"].rank(ascending=True)
        # 5. Rankear la capacidad de los almacenes
        opciones["rank_capacidad"] = opciones["capacidad"].rank(ascending=False)
        # 6. Calcular (Rankeo de precios * 0.2 + Rankeo de capacidad * 0.8)
        opciones["rank_total"] = (
            opciones["rank_precio"] * 0.2 + opciones["rank_capacidad"] * 0.8
        )
        # 7. Seleccionar el mejor almacen por departamento
        opciones = opciones.sort_values("rank_total", ascending=True)
        opcion_a_escoger = {
            "divipol": divipol,
            "ubicacion": opciones.iloc[0]["location"],
            "area": opciones.iloc[0]["area"],
            "capacidad": opciones.iloc[0]["capacidad"],
            "precio": opciones.iloc[0]["price"],
        }
        opcion_a_escoger = pd.DataFrame(opcion_a_escoger, index=[0])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=FutureWarning)
            opciones_de_almacenes_final = pd.concat(
                [opciones_de_almacenes_final, opcion_a_escoger]
            )
    # set divipol as index
    opciones_de_almacenes_final = opciones_de_almacenes_final.set_index("divipol")

    return opciones_de_almacenes_final


def procesar_municipios_completos(municipios, matriz_de_costos_final):
    # 1. Eliminar filas que no estén en la matriz de costos
    print(f"     Se eliminaron {len(municipios) - len(matriz_de_costos_final)} filas")
    municipios = municipios.loc[matriz_de_costos_final.index]
    # 2. tomar solo las columnas de interés
    columnas_de_interes = ["municipio", "departamento", "lat", "lon"] + [
        str(x) for x in range(1985, 2024)
    ]
    municipios_final = municipios[columnas_de_interes]
    return municipios_final


def procesar_matriz_de_distancias_completas(
    matriz_de_distancias, municipios, matriz_de_costos_final
):
    # 1. Eliminar filas y columnas que no estén en la matriz de costos
    matriz_de_distancias = matriz_de_distancias.loc[
        matriz_de_costos_final.index, matriz_de_costos_final.index
    ]
    # 2. Rectificar que no existan indices o nombres de columna duplicados
    matriz_de_distancias = matriz_de_distancias.loc[
        ~matriz_de_distancias.index.duplicated(keep="first")
    ]
    matriz_de_distancias = matriz_de_distancias.loc[
        ~matriz_de_distancias.columns.duplicated(keep="first")
    ]
    # 3. Rectificar que no existan valores nulos
    if matriz_de_distancias.isnull().sum().sum() > 0:
        assert matriz_de_distancias.isnull().sum().sum() == 0, "Existen valores nulos"
        matriz_de_distancias = matriz_de_distancias.dropna()
        matriz_de_distancias = matriz_de_distancias.loc[
            matriz_de_distancias.index, matriz_de_distancias.columns
        ]
        print("     Se eliminaron los valores nulos")
        # 4. Rectificar que no existan valores negativos
    assert (matriz_de_distancias < 0).sum().sum() == 0, "Existen valores negativos"
    # 5. Rectificar que no existan valores iguales a cero fuera de la diagonal principal
    assert (
        matriz_de_distancias.values[~np.eye(matriz_de_distancias.shape[0], dtype=bool)]
        == 0
    ).sum() == 0, "Existen valores iguales a cero fuera de la diagonal principal"
    # 6. Rectificar que la matriz tenga simetría
    assert len(matriz_de_distancias) == len(
        matriz_de_distancias.columns
    ), "La matriz no es simétrica"
    # 7. Rectificar que la diagonal principal sea cero
    assert (
        np.diag(matriz_de_distancias) == 0
    ).all(), "La diagonal principal no es cero"
    # 8. utilizar el valor máximo entre la distancia entre los municipios y la distancia en línea recta
    correccion = 0
    for i in matriz_de_distancias.index:
        for j in matriz_de_distancias.columns:
            dato_actual = matriz_de_distancias.loc[i, j]
            distancia = geopy.distance.distance(
                (municipios.loc[i, "lat"], municipios.loc[i, "lon"]),
                (municipios.loc[j, "lat"], municipios.loc[j, "lon"]),
            ).km
            if dato_actual < distancia:
                matriz_de_distancias.loc[i, j] = distancia
                correccion += 1
    if correccion > 0:
        print(f"     Se corrigieron {correccion} valores por distancia en línea recta")
    return matriz_de_distancias


def procesar_matriz_de_costos_completos(matriz_de_costos):
    # 1. Rectificar que no existan indices o nombres de columna duplicados
    matriz_de_costos = matriz_de_costos.loc[
        ~matriz_de_costos.index.duplicated(keep="first")
    ]
    matriz_de_costos = matriz_de_costos.loc[
        ~matriz_de_costos.columns.duplicated(keep="first")
    ]
    # 2. Rectificar que no existan valores nulos
    assert matriz_de_costos.isnull().sum().sum() == 0, "Existen valores nulos"
    # 3. Rectificar que no existan valores negativos
    assert (matriz_de_costos < 0).sum().sum() == 0, "Existen valores negativos"
    # 4. Rectificar que no existan valores iguales a cero fuera de la diagonal principal
    assert (
        matriz_de_costos.values[~np.eye(matriz_de_costos.shape[0], dtype=bool)] == 0
    ).sum() == 0, "Existen valores iguales a cero fuera de la diagonal principal"
    # 5. Rectificar que la matriz tenga simetría
    assert len(matriz_de_costos) == len(
        matriz_de_costos.columns
    ), "La matriz no es simétrica"
    # 6. Rectificar que la diagonal principal sea cero
    assert (np.diag(matriz_de_costos) == 0).all(), "La diagonal principal no es cero"
    return matriz_de_costos


In [16]:
# PARAMETROS
comida_per_capita = 0.00087617  # 0.87617 kg por persona
densidad_de_alimentos = 537 / 1000  # 537 kg por m3 (5 metros de altura)

# Crear la estructura de archivos
crear_estructura_de_archivos()
# Cargar los datos
matriz_de_costos, matriz_de_distancias, municipios, opciones_de_almacenes = (
    cargar_datos()
)
# Alistar los datos completos
matriz_de_costos, matriz_de_distancias, municipios, opciones_de_almacenes = (
    alistar_datos_completos(
        matriz_de_costos,
        matriz_de_distancias,
        municipios,
        opciones_de_almacenes,
        comida_per_capita,
        densidad_de_alimentos,
    )
)


Matriz de costos completada
     Se corrigieron 2 valores por distancia en línea recta
Matriz de distancias completada
     Se eliminaron 691 filas
Municipios completados
     No se encontró un almacen que cumpla la demanda para 18001
Opciones de almacenes completadas
